# Programming exercise 8: The AKLT model and matrix product states

Due on Monday, 24.12.2018, 23.59h

__This exercise sheet has been solved by:__ Mathieu Kaltschmidt and Carl von Randow

### Total: 100 points

In [1]:
# load standard libraries
import scipy.sparse as sp
import scipy.sparse.linalg as sLA
import numpy as np
import numpy.linalg as LA
import matplotlib.pyplot as plt
%matplotlib inline

#Plot settings
plt.style.use('seaborn-talk')
plt.rcParams["font.family"]='serif'
plt.rcParams["figure.figsize"][0] = 14
plt.rcParams["figure.figsize"][1] = 9

### Exercise 1: Solving the AKLT model with exact diagonalization (25 points)

This time we want to study a quantum many-body model for spins known as the AKLT model which was introduced in the 80's to explore the Haldane conjecture(see https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.59.799). It has a couple of remarkable features that will allow us to use it as playground for testing the basic ideas surrounding matrix product states. As additional practice on how to build general spin models and exact diagonalization, we will start by solving it here by numerical means and then compare the result to some theoretical predictions. You can use some of the code from the last exercise as a basis but make sure to modify the operators as now we are working with spin 1!  

As mentioned, this is a spin 1 model and it lives in a one dimensional chain with periodic boundary conditions (using the correct boundary condition is critical here) and the Hamiltonian is given by:

$$
H=J\sum_{i=0}^{N-1}\left(\frac{1}{3} +\frac{1}{2}S^{(i)}\cdot S^{(i+1)}+\frac{1}{6}(S^{(i)}\cdot S^{(i+1)})^2\right)
$$
Remenber that, due to the periodic boundary conditions, the Nth spin is identified with 0th spin. $S^{(i)}$ denotes the vector of spin sperators $(S_x^{(i)}, S_y^{(i)}, S_z^{(i)})$. It might be convenient to use the following representation for the spin their products:
$$
S^{(i)}\cdot S^{(i+1)} = S_z^{(i)} S_z^{(i+1)}+\frac{1}{2}\left(S_+^{(i)}S_-^{(i+1)}+S_-^{(i)}S_+^{(i+1)}\right)
$$
where the relevant spin 1 spin operators are $S_z=-|-1\rangle\langle-1| + |+1\rangle\langle+1|$, $S_+=\sqrt{2}(|+1\rangle\langle 0| + |0\rangle\langle -1|)$ and $S_- = S_+^\dagger$ (with $|s\rangle$ denoting the basis states in the $S_z$ eigenbasis).

Build the Hamiltonian in the standard product basis ($J$ is just a global energy scale and can be set to 1).

First you can test your implementation by calculating the ground state energy and making sure it is zero as this is the theoretical prediction. Another test is the magnetization on the z direction which should also be zero.

Once you are confident that your code is working you should compute the energy gap between ground and first excited state for a couple of system sizes $N$ and see how it behaves. Is the system gapped? This question is directly related to the Haldane conjecture.

Finally, compute the correlation function $\langle S_z^{(i)}S_z^{(i+r)} \rangle$ as a function of $r$ and compare to the theoretical result:
$$
\langle S_z^{(i)}S_z^{(i+r)}\rangle \propto \left(\frac{-1}{3}\right)^r
$$

It probably won't look so good as the exact diagonalization is not the best tool to study this but you should get some agreement.


Optional: Investigate what happend for open boundary conditions and see if there is any qualitative difference.

## Solution: 

In [2]:
#Predefined values
(hbar, J) = (1, 1)

In [3]:
SpinUp, SpinZero, SpinDown = hbar * np.eye(3)

#Building the Spin Operators as Sparse matrices
Sz = sp.csr_matrix(-np.outer(SpinDown, SpinDown) + np.outer(SpinUp, SpinUp))
SPlus = sp.csr_matrix(np.sqrt(2)*(np.outer(SpinUp, SpinZero) + np.outer(SpinZero, SpinDown)))
SMinus = sp.csr_matrix(np.transpose(SPlus))

In [4]:
#Building the Single Spin Operators
def SingleSpinOps(N):
    
    Sz_i = [sp.kron( sp.kron( sp.identity(3**(N-i-1)), Sz), sp.identity(3**i)) for i in range(N)] 
    SPlus_i = [sp.kron( sp.kron( sp.identity(3**(N-i-1)), SPlus), sp.identity(3**i)) for i in range(N)]
    SMinus_i = [sp.kron( sp.kron( sp.identity(3**(N-i-1)), SMinus), sp.identity(3**i)) for i in range(N)] 
    
    return np.array(Sz_i), np.array(SPlus_i), np.array(SMinus_i)

#Hamiltonian for the Transverse Field Ising model with periodic boundary conditions
def buildH(N, Sz_i, SPlus_i, SMinus_i):
    H = J * N/3 * sp.identity(3**N)
    for i in range(N):
        prod = Sz_i[i] @ Sz_i[(i+1)%N] + 1/2*( SPlus_i[i] @ SMinus_i[(i+1)%N] + SMinus_i[i] @ SPlus_i[(i+1)%N])
        H += 1/2 * prod + 1/6 * prod @ prod
    return H

### Ground state energy

In [5]:
#Some Test values for N
Nvals = np.arange(4,19,1, dtype = int)

for N in Nvals[:-6]:
    Sz_i, SPlus_i, SMinus_i = SingleSpinOps(N)
    
    #Calculating the ground state energy
    H = buildH(N, Sz_i, SPlus_i, SMinus_i)
    
    #Determining the energy of the ground state
    E0 = sLA.eigsh(H, k=3, which='SA')[0][0]
    print('The ground state energy for N = {} has been determined to'.format(N), E0)
    print()    

The ground state energy for N = 4 has been determined to -6.10935073118e-16

The ground state energy for N = 5 has been determined to 1.29006165559e-15

The ground state energy for N = 6 has been determined to 8.29302397038e-16

The ground state energy for N = 7 has been determined to -1.37935711616e-15

The ground state energy for N = 8 has been determined to -9.15561844506e-17

The ground state energy for N = 9 has been determined to -6.83838737381e-16

The ground state energy for N = 10 has been determined to -5.58727627126e-16

The ground state energy for N = 11 has been determined to -1.44627071694e-15

The ground state energy for N = 12 has been determined to -1.91428220117e-15



### Discussion:
These values can be considered as zero and therefore they show that our calculation of the ground state energy is valid.

### Energy gap for different $N$

In [ ]:
#Initialization
gap = []
#Calculating the gap for all the N values
for N in Nvals:
    Sz_i, SPlus_i, SMinus_i = SingleSpinOps(N)
    H = buildH(N, Sz_i, SPlus_i, SMinus_i)
    energies = sLA.eigsh(H, k=2,which="SA")[0]
    gap.append(energies[1]-energies[0])

In [ ]:
#Plotting the result
plt.plot(Nvals, gap, linestyle = ' ', color = 'darkred', marker = 'o')
plt.title(r'Energy gap between the ground state and the first excited state', size = 22)
plt.ylabel(r'Energy gap $\Delta E$', size = 20)
plt.xlabel(r'Chain size $N$', size = 20)
plt.grid(True, linestyle = 'dashed')

### Discussion:
As one would expect from the fact that we are observing a Spin-1 Heisenberg chain, there is an energy gap between the first excited state and the ground state. In the limit of large $N$ the value of the energy difference converges to a value $\sim 0.35 \ [AU] $

## Spin correlation function

In [ ]:
#Theoretical expectation
def SpinCorrelationTheo(const,r):
    return const * (-1/3)**r

#Numerical calculation
def SpinCorrelationNum(i,r,N):
    if i < 0 or i > N:
        raise ValueError('Index out of range')
    
    (Sz_i1, Sz_i2) = (SingleSpinOps(N)[0][i], SingleSpinOps(N)[0][i+r])
    corr = Sz_i1 @ Sz_i2
 
  #Hier fehlt noch was!        

### Discussion

Hier könnte ihre Interpretation stehen :-)

### Exercise 2: Entanglement in the phase diagram (25 points)

Now we are going to look at entanglement in the AKLT model but to make things more interesting we add a paramenter $\theta$ to the Hamiltonian:
$$
H=J\sum_{i=0}^{N-1}\left(\frac{cos(\theta)}{3 cos(\theta_{0})} +\frac{cos(\theta)}{2 cos(\theta_{0})}S^{(i)}\cdot S^{(i+1)}+\frac{sin(\theta)}{2 cos(\theta_{0})}(S^{(i)}\cdot S^{(i+1)})^2\right)
$$
where $\theta_0 = \text{arctan}(1/3)$ is a special angle that allow us to go back to the original Hamiltonian with $\theta=\theta_0$.


Study what happens to the gap of the system as you change $\theta$, moving away from $\theta_0$, and interpret your results. After this you should look at the entanglement of the ground state for different $\theta$ taking into account your conclusions about the gap. What happens to the entanglement entropy and the schmidt rank as $\theta$ changes?

Tip: a nice trick to compute partial traces is to use the function reshape() as it allows you to rewrite your state as a matrix in the way required for a Schmidt decomposition. After that all you need to do to get the entanglement spectrum is perform a svd (also available in the usual places) and look at the singular values.

## Solution:

In [ ]:
#Building the modified Hamiltonian
def buildH_mod(N, Sz_i, SPlus_i, SMinus_i, theta):
    #Initialization
    H = J * N * np.cos(theta)/(3 * np.cos(np.arctan(1/3))) * sp.identity(3**N)
    for i in range(N):
        prod = Sz_i[i] @ Sz_i[(i+1)%N] + 1/2 * (SPlus_i[i] @ SMinus_i[(i+1)%N] + SMinus_i[i] @ SPlus_i[(i+1)%N])
        H += np.cos(theta)/(2*np.cos(np.arctan(1/3)))* prod + np.sin(theta)/(6 * np.cos(np.arctan(1/3))) * prod @ prod
    return H

In [ ]:
thetaVals = np.arange(0,2*np.pi,np.pi/20)

#Comparing the result for different N
gap_all = []
states_all = []
for N in Nvals[:6]:
    Sz_i, SPlus_i, SMinus_i = SingleSpinOps(N)
    gap = []
    statesN = []
    #Trying different thetas
    for theta in thetaVals:
        H = buildH_mod(N, Sz_i, SPlus_i, SMinus_i, theta)
        energies, states = sLA.eigsh(H, k=3, which="SA")
        #Energy gap
        gap.append(energies[1] - energies[0])
        #Corresponding states
        statesN.append(states)
        
    #Storing all the results
    gap_all.append(gap)
    states_all.append(statesN)
    
    #Plotting the result
    plt.plot(thetaVals/np.pi, gap, label= 'gap for N = {}'.format(N))
    
#Setting up the Plot
plt.plot([np.arctan(1/3)/np.pi]*2, (-0.05,0.7), linestyle = 'dashed', color = 'darkred', 
         label = r'$\theta_0$ = arctan$\left(\frac{1}{3}\right)$')
plt.xlabel(r'$\theta$ [$\pi$]', size = 20)
plt.ylabel(r'Energy difference $\Delta E$', size = 20)
plt.ylim(-0.05,0.65)
plt.title(r'Modified energy gap as a function of $\theta$', size = 22)
plt.legend(fontsize='large', borderpad=1.5, borderaxespad=1.5)

In [ ]:
#Further analysis using the respective ground state

#Test case: N=3, Theta = 0
Psi = states_all[0][0][:,0] 

#Calculating the Schmidt rank 
def SchmidtRank(state):
    #Correct reshaping
    reshape = np.reshape(state,(A,B))
    
    #Obtain U, S and V matrix
    U, S, V = LA.svd(reshape)
    S = np.diag(S)
    
    #Counting non-zero elements
    for i in range(len(S)):
        rank = 0
        while np.abs(S[i]) > 2e-16: #float 64 accuracy
            rank += 1
    return rank
    
def EntanglementEntropy(state):
    #Correct reshaping
    reshape = np.reshape(state,(A,B))
    
    #Obtain U, S and V matrix
    U, S, V = LA.svd(reshape)
    S = np.diag(S)
    
    #Calculating the entropy
    for i in range(len(S)): #correct range?
        entropy = 0
        if S[i] < 2e-16: 
            raise ValueError('Negative value in log encountered!')
        entropy += S[i] * np.log(S[i]) #maybe S^2 ??
    return - entropy

### What happens to the Schmidt rank and the entropy for different values for $\theta$ ?

In [ ]:
#Determining the results for all theta values
schmidt_ranks = []
entropies = []

for theta in range(len(thetaVals)):
    #Here: N=3
    Psi = states_all[0][theta][:,0] 
    schmidt_ranks.append(SchmidtRank(Psi))
    entropy.append(EntanglementEntropy(Psi))    

In [ ]:
#Plotting the results for the Schmidt decomposition
plt.plot(thetaVals/np.pi, schmidt_ranks, label= 'Schmidt rank', color = 'darkred')

plt.xlabel(r'$\theta$ [$\pi$]', size = 20)
plt.ylabel(r'Corresponding Schmidt rank', size = 20)
plt.title(r'Impact on the Schmidt rank for varying $\theta$', size = 22)

In [ ]:
#Plotting the results for the entropy
plt.plot(thetaVals/np.pi, entropies, label= 'Entanglement entropy', color = 'darkred')

plt.xlabel(r'$\theta$ [$\pi$]', size = 20)
plt.ylabel(r'Entanglement entropy', size = 20)
plt.title(r'Impact on the Entanglement entropy for varying $\theta$', size = 22)

### Discussion:

Hier könnte Ihre Interpretation stehen!

### Exercise 3: MPS description (50 points)

Finally we will use the MPS machinery and explain why we chose to study the AKLT model (besides the fact that it is a pretty cool model :)). Usually for any given Hamiltonian it is necessary to apply a (complicated) procedure to find a good MPS aproximation for the ground states but the AKLT ground states is special in that it admits an exact MPS description so we can directly apply the tensor network techniques to it! The representation only requires bond dimension 2 and is given by the following 2x2 matrices:
$$
A_{+1}=\sqrt{\frac{2}{3}} \sigma_+\\
A_{0}=-\sqrt{\frac{1}{3}} \sigma_z\\
A_{-1}=-\sqrt{\frac{2}{3}} \sigma_-\\
$$

Show that this really gives the ground state by comparing it to the states you got from exact diagonalization. To do so you should use the expression for MPS ansatz to reconstruct the corresponding state in your basis
$$
|\psi_{\text{mps}}\rangle = \sum_{s_i} Tr(A_{s_0}A_{s_1}\dots A_{s_{N-1}})|s_0 s_1 \dots s_{N-1}\rangle
$$
and compute the overlap $\langle\psi_{\text{ed}}|\psi_{\text{mps}}\rangle$ with the exact diagonalzation ground state. Of course, this is not very smart as we are using our efficiennt MPS representation to reconstruct the inefficient ED representation, but it is a good sanity check nonetheless. 

You might notice a small deviation on the overlap. This comes from the fact that the MPS state is not normalized and you can see this by computing the norm directly from the matrices:
$$
\langle\psi_{\text{mps}}|\psi_{\text{mps}}\rangle = Tr(E^n)
$$
where
$$
E = \sum_s A_s\otimes A_s^*\\
$$
is the local matrix corresponding to the identity operator. To speed up this calculation you can diagonalize E so that you only need to compute powers of the eigenvales and not the actual matrix. Show that the deviation from 1 is small and vanishes exponentially for large systems.

Now, for the real benefits of the MPS, let's compute the correlation function $\langle S_z^{(i)}S_z^{(i+r)}\rangle$ from exercise 1. This can be done for very large systems by simply replacing two E matrices in the expression for the norm with
$$
E_z = \sum_{s,s'}\langle s|S_z|s'\rangle A_s\otimes A_{s'}^*
$$
which is the local matrix for $S_z$. Think about the correct position where the $E_z$ should go and use this to compute the correlation function. Compare the results to the theoretical prediction mentioned in the first exercise.

Optional: The AKLT model has a hidden order that manifests it self through a string operator:
$$
\langle S_z^{(i)}\,\exp\left[\sum_{i<k<j}i \pi S_z^{(k)}\right] \,  S_z^{(j)}\rangle
$$
Compute this order parameter and show that it maintains a constant value for $|i-j|>2$